In [22]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [24]:
import requests
import re
from html.parser import HTMLParser

In [27]:
url = "http://www.fromjapan.co.jp/japan/en/mercari/search/yohji/-/?sort_order=new"
header = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'}
request = urllib.request.Request(url, None, header)
response = urllib.request.urlopen(request)

html_bytes = response.read()
html = html_bytes.decode("utf-8")

In [29]:
print(html)

<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width,initial-scale=1">
    
    <!-- アプリ対応 -->
        <!-- SEO対策 -->
        <link rel="canonical" href="https://www.fromjapan.co.jp/japan/en/mercari/search/yohji/-/" id="cnl">
          <link rel="alternate" hreflang="jp" href="https://www.fromjapan.co.jp/japan/jp/mercari/search/yohji/-/" id="hrl-jp">
          <link rel="alternate" hreflang="en" href="https://www.fromjapan.co.jp/japan/en/mercari/search/yohji/-/" id="hrl-en">
          <link rel="alternate" hreflang="zh-Hans" href="https://www.fromjapan.co.jp/japan/cn/mercari/search/yohji/-/" id="hrl-cn">
          <link rel="alternate" hreflang="zh-Hant" href="https://www.fromjapan.co.jp/japan/tw/mercari/search/yohji/-/" id="hrl-tw">
          <link rel="alternate" hreflang="fr" href="https://www.fromjapan.co.jp/japan/fr/mercari/search/yohji/-/" id="hrl-fr">
 